In [1]:
#************************************************************************************
# Rezwan Matin
# Thesis B
# Filename: ML_MLP_RAVDESS_6.py
# Date: 3/20/20
#
# Objective:
# 26 MFCCs (mean) and 26 MFCCs (standard deviation), ZCR for Output Neurons for best SGD so far!
#
#*************************************************************************************

import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import librosa as rosa
import glob
import os
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import LearningRateScheduler

# Save directory path in 'path'
path = r'C:/Books/Texas State Books/Fall 2019/Thesis A/Corpus/Simulated/RAVDESS/All'

# Declare a dummy Numpy array (row vector)
result_array = np.empty([1,28])

# Create a list of audio file names 'file_list'
file_list = os.listdir(path)

i=0

for filename in file_list:
    
    # Read WAV file. 'rosa.core.load' returns sampling frequency in 'fs' and audio signal in 'sig'
    sig, fs = rosa.core.load(path + '/' + file_list[i], sr=None)
    
    # Calculate the average mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    avg_mfcc_feat = np.mean(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=13).T,axis=0)
    
    # Calculate the standard deviation of mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.std' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    std_mfcc_feat = np.std(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=13).T,axis=0)
    
    # Calculate the average zero crossing rate (utterance-level feature) using 'rosa.feat.zero_crossing_rate()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    zcross_feat = rosa.feature.zero_crossing_rate(sig)
    avg_zcross_feat = np.mean(rosa.feature.zero_crossing_rate(y=sig).T,axis=0)
    
    # Append the three 1D arrays into a single 1D array called 'feat'.
    feat0 = np.append(avg_mfcc_feat, std_mfcc_feat, axis=0)
    
    feat1 = np.append(feat0, avg_zcross_feat, axis=0)
    
    # Save emotion label from file name. 'path' contains directory's address, 'file_list' contains file name, and '\\' joins the two to form file's address
    label = os.path.splitext(os.path.basename(path + '/' + file_list[i]))[0].split('-')[2]
    
    # Create a new Numpy array 'sample' to store features along with label
    sample = np.insert(feat1, obj=27, values=label)
    
    result_array = np.append(result_array, sample)
    
    i+=1

ModuleNotFoundError: No module named 'tensorflow'

In [9]:
# Print out the 1D Numpy array
result_array

result_array.shape

# Convert 1D Numpy array to 2D array. Argument must be a Tuple. i+1 because we have i samples (audio files) plus a dummy row.
result_array = np.reshape(result_array, (i+1,-1))

# Delete first dummy row from 2D array
result_array = np.delete(result_array, 0, 0)

# Print final 2D Numpy array 
result_array.shape

(1440, 28)

In [10]:
df = pd.DataFrame(data=result_array)
# Label only the last (target) column
df = df.rename({27: "Emotion"}, axis='columns')
# Delete unnecessary emotion data (calm)
df.drop(df[df['Emotion'] == 2.0].index, inplace = True)
df['Emotion'].replace({1.0: 0.0, 3.0: 1.0, 4.0: 2.0, 5.0: 3.0, 6.0: 4.0, 7.0: 5.0, 8.0: 6.0}, inplace=True)
# Reset row (sample) indexing
df = df.reset_index(drop=True)
df.tail(12)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,Emotion
1236,-596.489909,60.752540,1.096842,10.348159,-2.365429,7.986745,-2.027191,-0.116491,-6.654691,-1.406857,...,18.495774,15.330220,11.818241,13.249881,9.333411,10.980157,8.693664,8.586983,0.066450,6.0
1237,-625.565085,37.601565,-3.692515,8.979277,-5.637381,1.969832,-14.028974,1.621135,-13.249778,-2.752983,...,14.422412,18.404797,10.772030,15.671242,9.325578,9.193904,9.050316,8.504878,0.100822,6.0
1238,-589.096402,81.593995,-0.673410,12.050125,10.394698,19.574634,-4.826030,2.336833,-1.092758,-2.186526,...,16.550125,20.497449,10.504126,12.649754,12.449414,11.125135,8.217028,10.183416,0.060662,6.0
1239,-567.861314,50.748331,-6.996325,2.507971,-4.691477,0.188368,-15.399068,-8.173095,-10.797020,-5.656966,...,15.191718,22.841168,14.320852,15.548943,9.642328,11.736763,9.461214,12.085072,0.071194,6.0
1240,-493.306198,48.233703,-3.028220,18.852342,-3.590502,9.721251,-2.724754,6.184319,-1.879220,2.796708,...,17.328371,15.802050,8.473895,10.908189,7.182592,11.731314,10.372954,9.465304,0.068974,6.0
1241,-615.406134,43.642912,-6.402692,12.012579,-11.143824,5.057568,-10.034647,9.767619,-5.317049,-4.267572,...,13.537131,16.787154,12.565453,12.114637,8.519819,9.465260,6.541300,8.739797,0.062682,6.0
1242,-696.193632,72.680527,6.986117,24.891391,2.902952,9.812980,1.004795,9.548923,-5.149150,-1.495172,...,13.503537,18.656377,12.994645,18.337063,13.666399,8.956852,11.963521,7.651668,0.053795,6.0
1243,-615.786554,53.243328,-18.664904,4.902306,-5.074936,2.717509,-12.459353,0.465174,-15.718720,-9.009732,...,12.934023,20.576538,8.970760,17.692649,11.438775,7.054764,9.191430,11.366741,0.073165,6.0
1244,-540.863667,61.330114,-9.318194,14.311654,0.700296,10.042510,-8.720129,-2.253887,-3.264636,-8.769154,...,14.379499,22.395955,12.870853,14.587431,12.936250,13.827785,7.848508,7.720903,0.069006,6.0
1245,-560.186503,44.504401,-14.639301,2.251491,-11.031190,5.346806,-10.163466,-6.553192,-9.599661,-7.118306,...,15.240751,19.054835,13.412277,15.255112,11.474124,14.669010,8.172784,9.568467,0.083883,6.0


In [11]:
df['Emotion'].value_counts()

# Balance the dataset for equal number of samples for each class.
# Separate majority and minority classes
df_minority = df[df.Emotion==0.0]
df_majority3 = df[df.Emotion==1.0]
df_majority4 = df[df.Emotion==2.0]
df_majority5 = df[df.Emotion==3.0]
df_majority6 = df[df.Emotion==4.0]
df_majority7 = df[df.Emotion==5.0]
df_majority8 = df[df.Emotion==6.0]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=192,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority3, df_majority4, df_majority5, df_majority6, df_majority7, df_majority8])
 
# Display new class counts
df_upsampled.Emotion.value_counts()

# Reset row (sample) indexing
df_upsampled = df_upsampled.reset_index(drop=True)

df_upsampled['Emotion'].value_counts()

# Extract target feature 'Emotion' in a vector y. Indexing from 0
y = df_upsampled.iloc[0:1344, 27].values
# Extract features 'buying' and 'safety' in a vector X. Indexing from 0
X = df_upsampled.iloc[0:1344, list(range(27))].values
print(y)

# Split data for training and testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

# Standardize the inputs
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

np.random.seed(123)

tf.random.set_seed(123)

# One-Hot Encode the classes
y_train_onehot = keras.utils.to_categorical(y_train)

# Create an object/instance 'model' for the 'Sequential()' class.
model = keras.models.Sequential()
model.add(
    keras.layers.Dense( units=53,
                input_dim=X_train_centered.shape[1],
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros', 
                activation='selu'))
model.add(
    keras.layers.Dense( units=45,
                input_dim=53,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))

model.add(
    keras.layers.Dense( units=54,
                input_dim=45,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))

model.add(
    keras.layers.Dense( units=y_train_onehot.shape[1],
                input_dim=54,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='softmax'))

# Define the learning rate schedule. This can then be passed as the learning rate for the optimizer.
lrate = keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=0.01, decay_steps=1000, decay_rate=0.8)

adam_optimizer = keras.optimizers.Adam(
                    learning_rate=lrate, beta_1=0.9, beta_2=0.999, epsilon=1e-06) #1e-06 gave better result than default value 1e-07

model.compile(optimizer=adam_optimizer,
                    loss='kullback_leibler_divergence')
                          
                            # cross-entropy: fancy name for logistic regression                        

# Train the MLP
history = model.fit(X_train_centered, y_train_onehot, batch_size=16, epochs=200, verbose=1, validation_split=0.1) # 90% training / 10% validation

y_train_pred = model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))

y_test_pred = model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]

print('Test accuracy: %.2f%%' % (test_acc * 100))

y_train_onehot.shape[1]

[0. 0. 0. ... 6. 6. 6.]
(1008, 27) (1008,)
(336, 27) (336,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 907 samples, validate on 101 samples
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/200
907/907 [==============================] - 1s 1ms/sample - loss: 1.7438 - val_loss: 1.5410
Epoch 2/200
907/907 [==============================] - 0s 314us/sample - loss: 1.5364 - val_loss: 1.4402
Epoch 3/200
907/907 [==============================] - 0s 271us/sample - loss: 1.4045 - val_loss: 1.4302
Epoch 4/200
907/907 [==============================] - 0s 249us/sample - loss: 1.4085 - val_loss: 1.3444
Epoch 5/200
907/907 [==============================] - 0s 219us/sample - loss: 1.3592 - val_loss: 1.3438
Epoch 6/200
907/907 [==============================] - 0s 270us/sample - loss: 1.

Epoch 144/200
907/907 [==============================] - 0s 215us/sample - loss: 0.0757 - val_loss: 1.5751
Epoch 145/200
907/907 [==============================] - 0s 214us/sample - loss: 0.0794 - val_loss: 1.5969
Epoch 146/200
907/907 [==============================] - 0s 216us/sample - loss: 0.0678 - val_loss: 1.5971
Epoch 147/200
907/907 [==============================] - 0s 212us/sample - loss: 0.0659 - val_loss: 1.6325
Epoch 148/200
907/907 [==============================] - 1s 668us/sample - loss: 0.0701 - val_loss: 1.6140
Epoch 149/200
907/907 [==============================] - 1s 567us/sample - loss: 0.0667 - val_loss: 1.6357
Epoch 150/200
907/907 [==============================] - 0s 225us/sample - loss: 0.0678 - val_loss: 1.6352
Epoch 151/200
907/907 [==============================] - 0s 240us/sample - loss: 0.0647 - val_loss: 1.6809
Epoch 152/200
907/907 [==============================] - 0s 224us/sample - loss: 0.0649 - val_loss: 1.6399
Epoch 153/200
907/907 [==============

7

In [9]:
#************************************************************************************
# Rezwan Matin
# Thesis B
# Filename: ML_MLP_RAVDESS_5.py
# Date: 2/23/20
#
# Objective:
# 26 MFCCs (mean) and 26 MFCCs (standard deviation), ZCR for BEST SGD so far!
#
#*************************************************************************************

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as rosa
import glob
import os
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
import tensorflow.keras as keras

# Save directory path in 'path'
path = r'C:/Books/Texas State Books/Fall 2019/Thesis A/Corpus/Simulated/RAVDESS+CREMA-D/All'

# Declare a dummy Numpy array (row vector)
result_array = np.empty([1,54])

# Create a list of audio file names 'file_list'
file_list = os.listdir(path)

i=0

for filename in file_list:
    
    # Read WAV file. 'rosa.core.load' returns sampling frequency in 'fs' and audio signal in 'sig'
    sig, fs = rosa.core.load(path + '/' + file_list[i], sr=None)
    
    # Calculate the average mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    avg_mfcc_feat = np.mean(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the standard deviation of mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.std' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    std_mfcc_feat = np.std(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=26).T,axis=0)
    
    # Calculate the average zero crossing rate (utterance-level feature) using 'rosa.feat.zero_crossing_rate()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    zcross_feat = rosa.feature.zero_crossing_rate(sig)
    avg_zcross_feat = np.mean(rosa.feature.zero_crossing_rate(y=sig).T,axis=0)
    
    # Append the three 1D arrays into a single 1D array called 'feat'.
    feat0 = np.append(avg_mfcc_feat, std_mfcc_feat, axis=0)
    
    feat1 = np.append(feat0, avg_zcross_feat, axis=0)
    
    # Save emotion label from file name. 'path' contains directory's address, 'file_list' contains file name, and '\\' joins the two to form file's address
    label = os.path.splitext(os.path.basename(path + '\\' + file_list[i]))[0].split('-')[2]
    
    # Create a new Numpy array 'sample' to store features along with label
    sample = np.insert(feat1, obj=53, values=label)
    
    result_array = np.append(result_array, sample)
    
    i+=1

# Print out the 1D Numpy array
result_array

result_array.shape

# Convert 1D Numpy array to 2D array
result_array = np.reshape(result_array, (i+1,-1))

# Delete first dummy row from 2D array
result_array = np.delete(result_array, 0, 0)

# Print final 2D Numpy array 
result_array.shape

df = pd.DataFrame(data=result_array)
# Label only the last (target) column
df = df.rename({53: "Emotion"}, axis='columns')
# Delete unnecessary emotion data (calm)
df.drop(df[df['Emotion'] == 2.0].index, inplace = True)
# Reset row (sample) indexing
df = df.reset_index(drop=True)
df.tail(12)

df['Emotion'].value_counts()

# Balance the dataset for equal number of samples for each class.
# Separate majority and minority classes
df_minority = df[df.Emotion==1.0]
df_majority3 = df[df.Emotion==3.0]
df_majority4 = df[df.Emotion==4.0]
df_majority5 = df[df.Emotion==5.0]
df_majority6 = df[df.Emotion==6.0]
df_majority7 = df[df.Emotion==7.0]
df_majority8 = df[df.Emotion==8.0]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=1463,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority3, df_majority4, df_majority5, df_majority6, df_majority7, df_majority8])
 
# Display new class counts
df_upsampled.Emotion.value_counts()

# Reset row (sample) indexing
df_upsampled = df_upsampled.reset_index(drop=True)

df_upsampled['Emotion'].value_counts()

df_minority_2 = df[df.Emotion==8.0]

# Upsample minority class
df_minority_upsampled_2 = resample(df_minority_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=1271,    # to match majority class
                                 random_state=123) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled_2, df_upsampled])
 
# Display new class counts
df_upsampled.Emotion.value_counts()

# Reset row (sample) indexing
df_upsampled = df_upsampled.reset_index(drop=True)

df_upsampled['Emotion'].value_counts()

# Extract target feature 'Emotion' in a vector y. Indexing from 0
y = df_upsampled.iloc[0:10241, 53].values
# Extract features 'buying' and 'safety' in a vector X. Indexing from 0
X = df_upsampled.iloc[0:10241, list(range(53))].values
print(y)

# Split data for training and testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

# Standardize the inputs
X_train_centered = (X_train - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

del X_train, X_test

print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

np.random.seed(123)

tf.set_random_seed(123)

# One-Hot Encode the classes
y_train_onehot = keras.utils.to_categorical(y_train)

# Create an object/instance 'model' for the 'Sequential()' class.
model = keras.models.Sequential()
model.add(
    keras.layers.Dense( units=53,
                input_dim=X_train_centered.shape[1],
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros', 
                activation='selu'))
model.add(
    keras.layers.Dense( units=54,
                input_dim=53,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))

model.add(
    keras.layers.Dense( units=44,
                input_dim=54,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='selu'))

model.add(
    keras.layers.Dense( units=y_train_onehot.shape[1],
                input_dim=44,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                activation='softmax'))

sgd_optimizer = keras.optimizers.SGD(
                    lr=0.01, decay=1e-7, momentum=0.9)

model.compile(optimizer=sgd_optimizer,
                    loss='categorical_crossentropy')
                          
                            # cross-entropy: fancy name for logistic regression
                            
# Train the MLP
history = model.fit(X_train_centered, y_train_onehot, batch_size=16, epochs=500, verbose=1, validation_split=0.1) # 90% training / 10% validation

y_train_pred = model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))

y_test_pred = model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]

print('Test accuracy: %.2f%%' % (test_acc * 100))

[8. 8. 8. ... 8. 8. 8.]
(7680, 53) (7680,)
(2561, 53) (2561,)
Train on 6912 samples, validate on 768 samples
Epoch 1/500
6912/6912 [==============================] - 2s 342us/sample - loss: 1.4722 - val_loss: 1.3706
Epoch 2/500
6912/6912 [==============================] - 2s 220us/sample - loss: 1.3624 - val_loss: 1.3475
Epoch 3/500
6912/6912 [==============================] - 1s 212us/sample - loss: 1.3180 - val_loss: 1.3001
Epoch 4/500
6912/6912 [==============================] - 2s 219us/sample - loss: 1.3047 - val_loss: 1.3130
Epoch 5/500
6912/6912 [==============================] - 2s 225us/sample - loss: 1.2930 - val_loss: 1.2798
Epoch 6/500
6912/6912 [==============================] - 2s 304us/sample - loss: 1.2818 - val_loss: 1.3483
Epoch 7/500
6912/6912 [==============================] - 2s 218us/sample - loss: 1.2722 - val_loss: 1.3231
Epoch 8/500
6912/6912 [==============================] - 2s 254us/sample - loss: 1.2641 - val_loss: 1.2495
Epoch 9/500
6912/6912 [============

6912/6912 [==============================] - 1s 213us/sample - loss: 0.9837 - val_loss: 1.1609
Epoch 151/500
6912/6912 [==============================] - 2s 220us/sample - loss: 0.9623 - val_loss: 1.1538
Epoch 152/500
6912/6912 [==============================] - 2s 350us/sample - loss: 0.9719 - val_loss: 1.2367
Epoch 153/500
6912/6912 [==============================] - 2s 230us/sample - loss: 0.9543 - val_loss: 1.3904
Epoch 154/500
6912/6912 [==============================] - 2s 235us/sample - loss: 0.9933 - val_loss: 1.1212
Epoch 155/500
6912/6912 [==============================] - 1s 215us/sample - loss: 0.9408 - val_loss: 1.1541
Epoch 156/500
6912/6912 [==============================] - 2s 218us/sample - loss: 0.9608 - val_loss: 1.2227
Epoch 157/500
6912/6912 [==============================] - 1s 215us/sample - loss: 0.9412 - val_loss: 1.1524
Epoch 158/500
6912/6912 [==============================] - 2s 219us/sample - loss: 0.9498 - val_loss: 1.2646
Epoch 159/500
6912/6912 [========

6912/6912 [==============================] - 1s 213us/sample - loss: 0.7924 - val_loss: 1.4878
Epoch 301/500
6912/6912 [==============================] - 1s 211us/sample - loss: 0.7850 - val_loss: 1.5149
Epoch 302/500
6912/6912 [==============================] - 2s 218us/sample - loss: 0.7960 - val_loss: 1.4335
Epoch 303/500
6912/6912 [==============================] - 2s 221us/sample - loss: 0.7905 - val_loss: 1.4347
Epoch 304/500
6912/6912 [==============================] - 2s 222us/sample - loss: 0.7851 - val_loss: 1.4916
Epoch 305/500
6912/6912 [==============================] - 2s 220us/sample - loss: 0.7657 - val_loss: 1.4681
Epoch 306/500
6912/6912 [==============================] - 1s 214us/sample - loss: 0.7556 - val_loss: 1.4219
Epoch 307/500
6912/6912 [==============================] - 2s 218us/sample - loss: 0.7683 - val_loss: 1.4832
Epoch 308/500
6912/6912 [==============================] - 2s 336us/sample - loss: 0.7799 - val_loss: 1.5129
Epoch 309/500
6912/6912 [========

6912/6912 [==============================] - 2s 340us/sample - loss: 0.7471 - val_loss: 1.9049
Epoch 451/500
6912/6912 [==============================] - 2s 260us/sample - loss: 0.6954 - val_loss: 2.0393
Epoch 452/500
6912/6912 [==============================] - 2s 220us/sample - loss: 0.7308 - val_loss: 1.8738
Epoch 453/500
6912/6912 [==============================] - 1s 212us/sample - loss: 0.7239 - val_loss: 1.7843
Epoch 454/500
6912/6912 [==============================] - 2s 226us/sample - loss: 0.7406 - val_loss: 2.1309
Epoch 455/500
6912/6912 [==============================] - 2s 226us/sample - loss: 0.7144 - val_loss: 1.9060
Epoch 456/500
6912/6912 [==============================] - 2s 227us/sample - loss: 0.7123 - val_loss: 1.9468
Epoch 457/500
6912/6912 [==============================] - 2s 224us/sample - loss: 0.7438 - val_loss: 2.0819
Epoch 458/500
6912/6912 [==============================] - 2s 223us/sample - loss: 0.7383 - val_loss: 2.2939
Epoch 459/500
6912/6912 [========